![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

***

Please copy this Jupyter notebook so that you are able to edit it.

Simply go to: File > Save a copy in Drive.

Once you have done that, you are good to go.

***

This tutorial is the interactive Jupyter notebook accompanying the [*Language Technology and Data Analysis Laboratory* (LADAL) tutorial *Converting PDFs to txt files with R*](https://ladal.edu.au/pdf2txt.html). 

***

**Preparation and session set up**

If you are using this notebook on Google Colab or your own computer and you have not already installed the R packages listed below, you need to install them. You can install them by running the code chunk below. 


In [ ]:
# install packages
install.packages("tesseract")
install.packages("dplyr")
install.packages("hunspell")


***



In [ ]:
# activate packages
library(tesseract)
library(dplyr)
library(hunspell)
# set tesseract engine
eng <- tesseract("eng")


Once you have initiated the session by executing the code shown above, you are good to go.


# OCR with tesseract

In this section, we use the `tesseract` package for optical character recognition (OCR) (see [here](https://cran.r-project.org/web/packages/tesseract/vignettes/intro.html) for more information and a more thorough tutorial on using the `tesseract` package). The `tesseract` package provides powerful OCR engine that supports over 100 languages. 

We start by creating a vector of paths to the pdf-files that we want to extract the text from.


In [ ]:
pdf0 <- "https://slcladal.github.io/data/PDFs/pdf0.pdf"
pdf1 <- "https://slcladal.github.io/data/PDFs/pdf1.pdf"

fls <- c(pdf0, pdf1)
# load
ocrs <- sapply(fls, function(x){
  # store name
  nm <- stringr::str_replace_all(x, ".*/(.*?).pdf", "\\1")
  # perform ocr
  x <- tesseract::ocr(x, engine = eng) %>%
    paste0(collapse = " ")
})
# show data
ocrs %>%
  substr(start=1, stop=500) %>%
  as.data.frame()


Although the results already look very promising, we want to see how we can combine automated spell-checking/correction with OCR as this is necessary when dealing with less pristine documents.

## Spell correction

In a first step, we write a function that loops over each text and checks which words occur in a English language dictionary (which we do not specify as it is the default). This spell checking makes use of the the `hunspell` package (see [here](https://cran.r-project.org/web/packages/hunspell/vignettes/intro.html) for more information). Hunspell is based on *MySpell* and is backward-compatible with *MySpell* and *aspell* dictionaries. This means that we can import and/or make use of many different language dictionaries and it is quite likely that the dictionaries for other languages may already available on your system!


In [ ]:
# create token list
tokens_ocr <- sapply(ocrs, function(x){
  x <- hunspell::hunspell_parse(x)
})
# show data
tokens_ocr %>%
  substr(start=1, stop=500) %>%
  as.data.frame()


In a next step, we can correct errors resulting from the OCR process, correct the errors and paste th texts back together (which is all done by the code chunk below). 



In [ ]:
# clean
clean_ocrtext <- sapply(tokens_ocr, function(x){
  correct <- hunspell::hunspell_check(x)
  x <- ifelse(correct == F, 
              x[hunspell::hunspell_check(x)],
              x)
  x <- paste0(x, collapse = " ")
})
# show data
clean_ocrtext %>%
  substr(start=1, stop=500) %>%
  as.data.frame()


## Saving the texts

To save the texts in txt-files on your disc, you can simply replace the predefined location (the data folder of your Rproject located by the string `here::here("data")` with the folder where you want to store the txt-files and then execute the code below. Also, we will name the texts (or the txt-files if you like) as *pdftext* plus their index number.


In [ ]:
# add names to txt files
names(txts) <- paste0("txt", 1:length(txts), sep = "")
# save result to disc
lapply(seq_along(txts), function(i)writeLines(text = unlist(txts[i]),
    con = paste(names(txts)[i],".txt", sep = "")))


We have reached the end of this tutorial and we hope that the tutorial helps you in performing OCR on your own pdfs.


[Back to LADAL](https://ladal.edu.au/pdf2txt.html)

***
